In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf=pyspark.SparkConf().setAppName('myapp').setMaster('local')


In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time
from pyspark.sql.functions import col, when

In [3]:
sc=pyspark.SparkContext(conf=conf)

In [4]:
from pyspark.sql import SQLContext
sql_c=SQLContext(sc)

In [5]:
spark=SparkSession(sc)

In [6]:
path_comics="/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/comics"
CHAR=sql_c.read.csv("/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/comics/characters.csv",header=True)
CHAR_TO=sql_c.read.csv("/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/comics/charactersToComics.csv",header=True)
COMICS=sql_c.read.csv("/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/comics/comics.csv",header=True)

### Ejercicio 1.- Al Dataframe que contiene los nombres de comics queremos agregar una columna que contenga los personajes a forma de array

In [13]:
COMICS.show()

+--------------------+--------------------+--------------------+--------------------+
|             comicID|               title|         issueNumber|         description|
+--------------------+--------------------+--------------------+--------------------+
|               16232|Cap Transport (20...|                  12|                null|
|               16248|Cap Transport (20...|                   9|                null|
|                4990| Halo Preview (2006)|                   0|                null|
|               21486|Ultimate X-Men (S...|                   9|                null|
|               58634|A Year of Marvels...|                   5|It’s Halloween in...|
|               37534|Magician: Apprent...|                  17|The electrifying ...|
|               58586|The Amazing Spide...|                  19|                null|
|               16241|Cap Transport (20...|                  20|                null|
|               25320|Halo Chronicles (...|           

In [14]:

CHAR_TO.show()
charid

+-------+-----------+
|comicID|characterID|
+-------+-----------+
|  16232|    1009220|
|  16232|    1010740|
|  16248|    1009220|
|  16248|    1009471|
|  16248|    1009552|
|  16248|    1009228|
|  21486|    1011109|
|  58634|    1010808|
|  58634|    1009515|
|  16241|    1009220|
|  16241|    1009252|
|  16241|    1009535|
|  16241|    1010899|
|  21479|    1011109|
|  21468|    1011010|
|  15808|    1011010|
|  17486|    1009175|
|  17486|    1009243|
|  17486|    1009257|
|  17486|    1009310|
+-------+-----------+
only showing top 20 rows



In [15]:
CHAR.show()

+-----------+--------------------+
|characterID|                name|
+-----------+--------------------+
|    1009220|     Captain America|
|    1010740|      Winter Soldier|
|    1009471|           Nick Fury|
|    1009552|        S.H.I.E.L.D.|
|    1009228|       Sharon Carter|
|    1011109|    X-Men (Ultimate)|
|    1010808|Hawkeye (Kate Bis...|
|    1009515|            Punisher|
|    1009252|          Crossbones|
|    1009535|           Red Skull|
|    1010899|            Spitfire|
|    1011010|Spider-Man (Ultim...|
|    1009175|               Beast|
|    1009243|            Colossus|
|    1009257|             Cyclops|
|    1009310|          Emma Frost|
|    1009508|         Kitty Pryde|
|    1009718|           Wolverine|
|    1009726|               X-Men|
|    1009359|               Hydra|
+-----------+--------------------+
only showing top 20 rows



In [112]:
ej_1_f=COMICS.join(CHAR_TO,on=['comicID'],how='left').join(CHAR,on=['characterID'],how='left')
ej_1_f.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet/comics',format='parquet')

## Dado el archivo players_20.csvn nuestro coach Ramón necesita saber 
#### 1.-Cuales son los 20 jugadores de menos de 23 años que tienen más potencial
#### 2.-Cuales son los equipos top 20 con el promedio (overall) más alto
#### 3.-De los 20 equipos anteriores nuestro coach también quiere saber cuáles son los 3 porteros y los 3 delanteros con mejor (overall)
#### 4.-Rankear a los jugadores por nacionalidad de tal forma que identifiquemos a los 5 mejores de cada país.

In [10]:
FIFA=sql_c.read.csv("/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/fifa/players_20.csv",header=True)

In [39]:
FIFA.columns

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality',
 'club',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'team_position',
 'team_jersey_number',
 'loaned_from',
 'joined',
 'contract_valid_until',
 'nation_position',
 'nation_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'player_traits',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_re

In [113]:
ej_1=FIFA.select('short_name','potential','age')
ej_1_r=ej_1.filter(ej_1.age < 23).orderBy(desc('potential')).limit(20)
ej_1_r.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet/fifa/ej1',format='parquet')

In [124]:
ej_2=FIFA.select('club','overall','team_position','short_name')
ej_21=ej_2.withColumn("overall",col("overall").cast(IntegerType()))

ej_2_r=ej_21.groupBy('club').agg(avg('overall').alias('avg_overall')).orderBy(desc('avg_overall')).where(col("avg_overall") >= 75.84848484848484)
ej_2_r.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet/fifa/ej2',format='parquet')
equipos=ej_2_r.toPandas()['club'].unique()

In [42]:
k=0
for i in equipos:
    ej_31=ej_2.filter(ej_2.club.contains(i)|ej_2.team_position.contains('W')|ej_2.team_position.contains('F'))
    ej_32=ej_2.filter(ej_2.club.contains(i)|ej_2.team_position.contains('GK'))
    
    ej_31=ej_31.orderBy(desc('potential')).limit(3)
    ej_32=ej_32.orderBy(desc('potential')).limit(3)
    ej_33=ej_32.union(ej_31)
    
    if k==0:
        ej_34=ej_33
        
    else:
        ej_34=ej_34.union(ej_33)
    k=k+1

In [116]:
ej_34.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet/fifa/ej3',format='parquet')

In [117]:
ej_4=FIFA.select('short_name','nationality','potential')
pais=ej_4.toPandas()['nationality'].unique()

k=0
for i in range(0,20):
    ej_41=ej_4.filter(ej_4.nationality.contains(pais[i]))
    ej_41=ej_41.orderBy(desc('potential')).limit(5)
    
    if k==0:
        ej_42=ej_41
        
    else:
        ej_42=ej_42.union(ej_41)
    k=k+1

In [119]:
ej_42.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet/fifa/ej4',format='parquet')

### Ejercicio 3 Filtrando por los tipos de pokemon(type1) fire y water es necesario calcular el promedio de cada una de las siguientes columnas: sp_attack, sp_defense y speed; de tal forma que el dataframe resultante muestre los siguientes datos: generation, avg_sp_attack_water, avg_sp_attack_fire, avg_sp_defense_water, avg_sp_defense_fire, avg_speed_water, avg_speed_fire

In [8]:
POKEMON=sql_c.read.csv("/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/input/csv/pokemon/PokemonData.csv",header=True)

In [108]:
ej_1_u=POKEMON.select('type1','sp_attack','sp_defense','speed','generation')

ej_1_u=ej_1_u.withColumn('sp_attack',col("sp_attack").cast(IntegerType()))
ej_1_u=ej_1_u.withColumn('sp_defense',col("sp_defense").cast(IntegerType()))
ej_1_u=ej_1_u.withColumn('speed',col("speed").cast(IntegerType()))

ej_1_fire=ej_1_u.filter(ej_1_u.type1.contains('fire'))
ej_1_water=ej_1_u.filter(ej_1_u.type1.contains('water'))

ej_1_fire=ej_1_fire.groupBy('type1').avg('sp_attack','sp_defense','speed')
ej_1_fire=ej_1_fire.select('type1',col("avg(sp_attack)").alias("avg_sp_attack_fire"),col("avg(sp_defense)").alias("avg_sp_defense_fire"),col("avg(speed)").alias("avg_speed_fire"))


ej_1_water=ej_1_water.groupBy('type1').avg('sp_attack','sp_defense','speed')
ej_1_water=ej_1_water.select('type1',col("avg(sp_attack)").alias("avg_sp_attack_water"),col("avg(sp_defense)").alias("avg_sp_defense_water"),col("avg(speed)").alias("avg_speed_water"))
ej_12=ej_1_fire.join(ej_1_water,on=['type1'],how='outer')
ej_12.show()

+-----+------------------+-------------------+-----------------+-------------------+--------------------+------------------+
|type1|avg_sp_attack_fire|avg_sp_defense_fire|   avg_speed_fire|avg_sp_attack_water|avg_sp_defense_water|   avg_speed_water|
+-----+------------------+-------------------+-----------------+-------------------+--------------------+------------------+
|water|              null|               null|             null|  74.06140350877193|   71.79824561403508|63.921052631578945|
| fire| 87.73076923076923|  71.53846153846153|73.34615384615384|               null|                null|              null|
+-----+------------------+-------------------+-----------------+-------------------+--------------------+------------------+



In [110]:
ej_12.write.save('/home/bma/Desktop/examen_diagnostico_datio-master/src/main/resources/output/parquet',format='parquet')